# Sequence Analysis with Python

Author: Riccardo Maria Pesce (riccardompesce@gmail.com)

Contact: Veli MÃƒÆ’Ã‚Â¤kinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [ ]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [ ]:
def dna_to_rna(s):
    to_dict = {"T": "U", "A": "A", "G": "G", "C": "C"}

    return "".join([to_dict[ch] for ch in s])
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

### Idea of solution

Trascribing a DNA sequence into RNA can be achieved by replacing each "T" (thymine) with "U" (uracil). As the problem requested, let's use a `dict` to store DNA-RNA correspondences, and with the aid of the `join` method, we can build up the RNA string, by first mapping in a list each character of the DNA string.

### Discussion

As we can see from the output, and comparing the result with an online DNA to RNA transcriber, we can see that our function works correctly.t two lines of code, we achieved a good result in converting DNA to RNA, as we can see from above output.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [ ]:
# Importing necessary libraries to implement get_dict()
import urllib, re

from bs4 import BeautifulSoup

def get_dict():
    # Parsing the data and turning it into a BeautifulSoup object
    page_url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"

    page = urllib.request.urlopen(page_url)
    soup = BeautifulSoup(page, "html.parser")

    # Parsing the actual content we are interested, i.e. the table between <pre></pre> tags
    pre_content = soup.find("pre")
    text_content = pre_content.text.strip()

    # Extracting the informations we are interested in
    dna_seqs = re.findall(r"[A-Z][A-Z][A-Z]", text_content)
    amino_acids = re.findall(r" ([A-Z]|[*]) ", text_content)

    # Returning the dict result
    return dict(zip(dna_seqs, amino_acids))
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

### Idea of solution
By using `urllib` and `BeautifulSoup`, we are going to parse the table in the given web_page. 
We are going to extract the content of the `pre` HTML tag and turn it into a long string, stripped from the whitespace characters.
By using RegEx, we are going to parse from such long string the relevant info, and we are going to build the requested dict from them.
In the notebook `scrape.ipynb` I wrote a full walkthrough of the code.

### Discussion
As we can see, the solution works. In a few lines of code, we were able to parse the web page and the requested informations. However, we used a third party library and we sacrificed clarity by using regex. But that is not a great issue.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [ ]:
def get_dict_list():
    d = get_dict()
    reverse = {}

    for v, k in d.items():
        if k not in reverse.keys():
            reverse[k] = []

        reverse[k].append(v)

    return reverse
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

### Idea of solution

The idea is simple: reversing the dictionary, but each value will be a list, since an amino acid can be a translation of different codons. The code is quite clear: for each key-value pair, if value is not present in the dict we want to return, we create that entry with an empty list which will later be filled.

### Discussion

The code works well. One thing that might be adjusted but for sake of order I won't really do is the fact that one-element lists can be turned into single elements.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [6]:

def rna_to_prot(s):
    d = get_dict()

    # Let's split s every 3 character so we can use our dict to convert each single sequence, and then we will join it
    splitted = [s[i:i + 3] for i in range(0, len(s), 3)]

    return "".join([d[seq] for seq in splitted])

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

The idea is to split the sequence `s` into sub-sequences of length 3 each, and map these sub-sequences using the dictionary returned by `get_dict()` into proteins.

### Discussion

Code works, but it can be improved a little bit in clarity, for example defining the joined list outside of the method call.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function.

In [7]:

def get_probability_dict():
    # Parsing the data and turning it into a BeautifulSoup object
    page_url = "http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N"

    page = urllib.request.urlopen(page_url)
    soup = BeautifulSoup(page, "html.parser")

    # Parsing the actual content we are interested, i.e. the table between <pre></pre> tags
    pre_content = soup.find("pre")
    text_content = pre_content.text.strip()

    # Extracting the informations we are interested in
    dna_seqs = re.findall(r"[A-Z][A-Z][A-Z]", text_content)
    freqs = re.findall(r" (\d+\.\d) \(", text_content)
    amino = re.findall(r" ([A-Z]|\*) ", text_content)

    # Calculating fractions from frequencies
    probs = [float(freq) / 1000 for freq in freqs]

    # Calculating cumulative probability for each amino
    amino_prob = {}
    for i, a in enumerate(amino):
        if a not in amino_prob.keys():
            amino_prob[a] = 0
        
        amino_prob[a] += probs[i]

    # Calculating the actual probabilities so that for each amino-acid the sequences probabilities sum up to 1
    dna_probs = dict(zip(dna_seqs, probs))
    dna_amino = dict(zip(dna_seqs, amino))
    seqs_probs = {}
    for dna, amino in dna_amino.items():
        seqs_probs[dna] = dna_probs[dna] / amino_prob[amino]

    # Returning the dict result
    return seqs_probs
    
if __name__ == '__main__':
    codon_to_prob = get_probability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.433393	AAC: 0.529086	AAG: 0.566607	AAU: 0.470914	ACA: 0.283835	ACC: 0.355263
ACG: 0.114662	ACU: 0.246241	AGA: 0.215168	AGC: 0.240444	AGG: 0.211640	AGU: 0.149199
AUA: 0.169300	AUC: 0.469526	AUG: 1.000000	AUU: 0.361174	CAA: 0.264516	CAC: 0.580769
CAG: 0.735484	CAU: 0.419231	CCA: 0.276596	CCC: 0.324059	CCG: 0.112930	CCU: 0.286416
CGA: 0.109347	CGC: 0.183422	CGG: 0.201058	CGU: 0.079365	CUA: 0.071856	CUC: 0.195609
CUG: 0.395210	CUU: 0.131737	GAA: 0.422741	GAC: 0.535181	GAG: 0.577259	GAU: 0.464819
GCA: 0.227994	GCC: 0.399711	GCG: 0.106782	GCU: 0.265512	GGA: 0.250000	GGC: 0.336364
GGG: 0.250000	GGU: 0.163636	GUA: 0.116969	GUC: 0.238880	GUG: 0.462932	GUU: 0.181219
UAA: 0.294118	UAC: 0.556364	UAG: 0.235294	UAU: 0.443636	UCA: 0.150432	UCC: 0.218249
UCG: 0.054254	UCU: 0.187423	UGA: 0.470588	UGC: 0.543103	UGG: 1.000000	UGU: 0.456897
UUA: 0.076846	UUC: 0.535620	UUG: 0.128743	UUU: 0.464380


### Idea of solution

The function will be similar to `get_dict()` with the only difference being the field we are parsing this time, which is the frequency in the fourth column. We will map the parsed probabilities into `float`, since they're initially parsed into strings.
The function will then calculate the relative (for each amino-acid) percentages one the normalized frequencies. Once that is finished, the probabilities are calculated for each amino.

### Discussion

As we can see from the output, the function worked perfectly.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [8]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        prob_dict = get_probability_dict()
        prot_to_rna = get_dict_list()

        most_prob_seq = []

        for prot in s:
            prob_seqs = prot_to_rna[prot]
            most_prob = prob_seqs[0]

            for seq in prob_seqs:
                if prob_dict[seq] > prob_dict[most_prob]:
                    most_prob = seq

            most_prob_seq.append(most_prob)

        return "".join(most_prob_seq)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The class method `convert()` maps each protein sequence into the most likely RNA sequence by first getting the probabilities and protein-to-rna dictionaries, and then for each protein in the sequence s, we get the one with the highest probability.

### Discussion

Using the [translation tool](http://biomodel.uah.es/en/lab/cybertory/analysis/trans.htm), we translate the result given by the function, and we can see it is exactly the protein sequence given.



Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [9]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    cum_dist = {}
    start = 0.00

    for event, prob in dist.items():
        cum_dist[event] = [start, np.round(prob + start, 2)]
        start = np.round(start + prob, 2)

    r = np.random.uniform()

    for event in cum_dist.keys():
        if r >= cum_dist[event][0] and r < cum_dist[event][1]:
            return event

    

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(12)))

A, C, C, C, A, C, C, T, C, C, T, A


### Idea of solution

The function will first calculate the cumulative distribution, including for each protein the beginning and the end of the "bin" to which such protein belongs to. Then samples uniformly a random number from 0 to 1, and check in which bin it falls, returning the relative protein.

### Discussion

The function samples according to the proteins own probabilities, therefore it is correct

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        probs_dict = get_probability_dict()
        prot_to_rna = get_dict_list()

        rand_seq = []

        for prot in s:
            translations = prot_to_rna[prot]
            probs = [probs_dict[seq] for seq in translations]
            
            dist = dict(zip(translations, probs))

            rand_seq.append(random_event(dist))

        return "".join(rand_seq)


        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUAACUCCUAUUCAGAAUAGGGCC


### Idea of solution

The method is similiar to the previous one, with the sole exception that instead of using the most probable RNA, the function uses the previously defined random event to sample from the cumulative distribution of each RNA.

### Discussion

The function seems to sample according to the amino-acids distributions.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [11]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    i = 0

    while i + k < len(s):
        freqs = {}

        for prot in s[i:i + k]:
            if prot not in freqs.keys():
                freqs[prot] = 0
            
            freqs[prot] += 1

        yield freqs 

        freqs[s[i]] -= 1

        if s[i + k] not in freqs.keys():
            freqs[s[i + k]] = 0

        freqs[s[i + k]] += 1

        i += 1
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'T': 1, 'C': 3}
{'C': 3, 'G': 1}
{'C': 2, 'G': 1, 'A': 1}
{'C': 2, 'G': 1, 'A': 1}
{'G': 2, 'A': 1, 'C': 1}
{'A': 1, 'C': 1, 'G': 2}
{'C': 2, 'G': 2}
{'G': 2, 'C': 2}
{'G': 1, 'C': 2, 'T': 1}
{'C': 2, 'T': 2}
{'C': 1, 'T': 2, 'G': 1}
{'T': 2, 'G': 1, 'C': 1}


### Idea of solution

For each substring of size `k`, the function is going to compute how many times each single nucletoide is present, and will store this information in a dictionary, whose key is the nucletoide and the value is the frequency.

### Discussion

The functions showed the correct frequencies for each window.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [12]:
def context_list(s, k):
    i = 0
    contextes = {}

    while i + k < len(s):
        context = s[i:i + k]
        if context in contextes.keys():
            contextes[context] = "".join(list(contextes[context]) + [s[i + k]])
        else:
            contextes[context] = s[i + k]
        i += 1

    return contextes
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The idea is to iterate through each k-mer, and store in a dict whose keys are the former the string with each context.

### Discussion

The output of the function satisfies the requirements.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [13]:
def context_probabilities(s, k):
    context_probs = {}

    for k, v in context_list(s, k).items():
        context_probs[k] = {ch: v.count(ch) / len(v) for ch in v}

    return context_probs

    
if __name__ == '__main__':
    k = 0
    s = "ATGATATCATCGACGATGTAG"
    print(context_probabilities(s, k))
    k = 2
    print(context_probabilities(s, k))

{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


### Idea of solution

The function should count the total number of contextes together with contextes themselves and then it should convert these frequencies into probabilities, by dividing the number of occurrences of each element by the total number of contextes.

### Discussion

The function does what said above, and the output seems coherent with the requirements of the function.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [14]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        np.random.seed(seed)
        
        seq = []
        i = 0

        while i < n:
            while i < k:
                seq.append(random_event(zeroth,))
                i += 1
            
            ctx = "".join(seq[i - 2:i])
            seq.append(random_event(kth[ctx]))
            
            i += 1

        return "".join(seq)
            


if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

TGTATGATGA


### Idea of solution

The function is required to initialize the given seed and then generate the first k element using the zeroth dict, proceeding to generating the rest using the given kth dict of dicts.

### Discussion

The function seems to generate a legit sequence number.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [15]:
from itertools import product

def context_pseudo_probabilities(s, k):
    symbols = "".join(set(s))
    context_probs = context_probabilities(s, k)
    context_pseudo_probs = {}
    k_mers = ["".join(t) for t in product(symbols, repeat=k)]
    print(k_mers)

    for k_mer in k_mers:
        if k_mer not in s[:-1]:
            context_pseudo_probs[k_mer] = {ch: symbols.count(ch) / len(symbols) for ch in symbols}
        else:
            context_pseudo_probs[k_mer] = context_probs[k_mer]

    return context_pseudo_probs

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

['CC', 'CG', 'CT', 'CA', 'GC', 'GG', 'GT', 'GA', 'TC', 'TG', 'TT', 'TA', 'AC', 'AG', 'AT', 'AA']
['']
zeroth: {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}
CC: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}
CG: {'A': 1.0}
CT: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}
CA: {'T': 1.0}
GC: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}
GG: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}
GT: {'A': 1.0}
GA: {'T': 0.6666666666666666, 'C': 0.3333333333333333}
TC: {'A': 0.5, 'G': 0.5}
TG: {'A': 0.5, 'T': 0.5}
TT: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}
TA: {'T': 0.5, 'G': 0.5}
AC: {'G': 1.0}
AG: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}
AT: {'G': 0.4, 'A': 0.2, 'C': 0.4}
AA: {'C': 0.25, 'G': 0.25, 'T': 0.25, 'A': 0.25}

 ATAGATCGATCATCATGTAT


### Idea of solution

The function can leverage the previous function for the present k-mers, while it should initialize to uniform probabilities the absent k-mers.

### Discussion

The function output is coherent to the description.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [16]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1
        i = 0

        while i + self.k < len(s):
            if i < self.k:
                ctx = s[i]
                prob *= zeroth[ctx]
            else:
                ctx = s[i - 2:i]
                nxt = s[i]
                k_mer = kth[ctx]
                prob *= k_mer[nxt]

            i += 1


        return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

['CC', 'CG', 'CT', 'CA', 'GC', 'GG', 'GT', 'GA', 'TC', 'TG', 'TT', 'TA', 'AC', 'AG', 'AT', 'AA']
['']
Probability of sequence ATGATATCATCGACGATGTAG is 2.257495590828924e-06


### Idea of solution

The function should return the product of the probabilities for each symbol defined in the zeroth (before reaching the kth element) and in the kth.

### Discussion

The function seems to return the correct probability.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [17]:
class MarkovLog(object):
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        prob = 1
        i = 0

        while i + self.k < len(s):
            if i < self.k:
                ctx = s[i]
                prob *= zeroth[ctx]
            else:
                ctx = s[i - 2:i]
                nxt = s[i]
                k_mer = kth[ctx]
                prob *= k_mer[nxt]

            i += 1


        return np.log2(prob)
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

['CC', 'CG', 'CT', 'CA', 'GC', 'GG', 'GT', 'GA', 'TC', 'TG', 'TT', 'TA', 'AC', 'AG', 'AT', 'AA']
['']
Log probability of sequence ATGATATCATCGACGATGTAG is -18.756845399379042


### Idea of solution

The function is the same as the previous, except that it now returns the base-2 log probability.

### Discussion

The function returns the correct log probability, which should go from minus infinity to zero.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [18]:
def better_context_probabilities(s, k):
    contextes = []

    concatenated = context_pseudo_probabilities(s, k)
    for _, v in concatenated.items():
        contextes += [ctx for ctx in v.keys()]

    print(contextes)

    return {ch: s.count(ch) / len(s) for ch in set(contextes)}

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

['CC', 'CG', 'CT', 'CA', 'GC', 'GG', 'GT', 'GA', 'TC', 'TG', 'TT', 'TA', 'AC', 'AG', 'AT', 'AA']
['C', 'G', 'T', 'A', 'A', 'C', 'G', 'T', 'A', 'T', 'C', 'G', 'T', 'A', 'C', 'G', 'T', 'A', 'A', 'T', 'C', 'A', 'G', 'A', 'T', 'C', 'G', 'T', 'A', 'T', 'G', 'G', 'C', 'G', 'T', 'A', 'G', 'A', 'C', 'C', 'G', 'T', 'A']
C: 0.14285714285714285
G: 0.23809523809523808
A: 0.3333333333333333
T: 0.2857142857142857


### Idea of solution

The function should return the probability of a symbol from the single context probabilities.

### Discussion

Results seem coherent.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [19]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k

    def generate(self, n, seed=None):
        np.random.seed(seed)

        seq = []
        freqs = better_context_probabilities(self.s, self.k)

        input_values = np.array(list(freqs.keys()))
        input_probs = np.array(list(freqs.values()))

        return "".join(np.random.choice(a=input_values, size=n, p=input_probs))
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

['CC', 'CG', 'CT', 'CA', 'GC', 'GG', 'GT', 'GA', 'TC', 'TG', 'TT', 'TA', 'AC', 'AG', 'AT', 'AA']
['C', 'G', 'T', 'A', 'A', 'C', 'G', 'T', 'A', 'T', 'C', 'G', 'T', 'A', 'C', 'G', 'T', 'A', 'A', 'T', 'C', 'A', 'G', 'A', 'T', 'C', 'G', 'T', 'A', 'T', 'G', 'G', 'C', 'G', 'T', 'A', 'G', 'A', 'C', 'C', 'G', 'T', 'A']
CTATTAACGA


### Idea of solution

Instead of concatenating, we compute the frequency of each symbol and use that to predict the sequence.

### Discussion

The function seems to work well.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [20]:
def kmer_index(s, k):
    kmer_ds = {}
    i = 0

    while i + k < len(s):
        kmer = s[i:i + k]
        
        if kmer not in kmer_ds.keys():
            kmer_ds[kmer] = []

        kmer_ds[kmer].append(i)

        i += 1                

    return kmer_ds

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17]}


### Idea of solution

The function iterates over each kmer and stores the index of each occurrence in the right dict key.

### Discussion

The function returns the correct indexes for each k-mer in the test sequence.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [21]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    c_probs = {}

    for codon in product(set(rna), repeat=3):
        c_seq = "".join(codon)
        if c_seq in rna[:-1]:
            kmer = kmer_index(rna, 3)
            c_probs[c_seq] = len(kmer[c_seq]) / len(rna)
        else: 
            c_probs[c_seq] = 0 

    print(c_probs)

    return c_probs
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return np.sum(np.array([p[x] * np.log(p[x] / q[x]) for x in p.keys()])) if 0 not in [q[x] for x in p.keys()] else np.inf

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    rna = ProteinToRandomRNA().convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    assert rna_to_prot(rna) == protein
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rna) # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {}
    i = 0
    table_probs = get_probability_dict()

    while i + 3 < len(rna):
        kmer = rna[i:i + 3]
        cp_orig[kmer] = table_probs[kmer]

        i += 1
    
    # Create a completely random RNA sequence and get the codon probabilities
    rand_rna = ProteinToRandomRNA().convert(protein)
    cp_uniform = codon_probabilities(rand_rna) # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

{'CCC': 0.0128, 'CCG': 0.009333333333333334, 'CCA': 0.0201, 'CCU': 0.014033333333333333, 'CGC': 0.0082, 'CGG': 0.0096, 'CGA': 0.011766666666666667, 'CGU': 0.0074, 'CAC': 0.017566666666666668, 'CAG': 0.019066666666666666, 'CAA': 0.0185, 'CAU': 0.0209, 'CUC': 0.010166666666666666, 'CUG': 0.021366666666666666, 'CUA': 0.013533333333333333, 'CUU': 0.012, 'GCC': 0.015166666666666667, 'GCG': 0.008466666666666667, 'GCA': 0.0175, 'GCU': 0.013966666666666667, 'GGC': 0.0158, 'GGG': 0.015633333333333332, 'GGA': 0.020066666666666667, 'GGU': 0.013966666666666667, 'GAC': 0.0177, 'GAG': 0.015433333333333334, 'GAA': 0.021766666666666667, 'GAU': 0.020133333333333333, 'GUC': 0.009933333333333334, 'GUG': 0.0207, 'GUA': 0.012366666666666666, 'GUU': 0.012266666666666667, 'ACC': 0.017566666666666668, 'ACG': 0.010933333333333333, 'ACA': 0.0205, 'ACU': 0.015866666666666668, 'AGC': 0.013233333333333333, 'AGG': 0.013466666666666667, 'AGA': 0.019233333333333335, 'AGU': 0.014366666666666666, 'AAC': 0.0160333333333

### Idea of solution

The function `codon_probabilities()` computes the probabilities of each codon based on their frequencies inside the sequence. The KL function is just the implementation of the KL entropy criterion.

### Discussion

We generated a random RNAs from the same protein. We can see that the probability distribution that better approximates the original distribution is the uniform one.

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [22]:
from sklearn.preprocessing import normalize

def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    stat_dist = []
    
    for dist in transition:
        t_dist = dist.T 
        t_tran = transition.T 

        print(np.matmul(t_tran, t_dist))

        w, v = np.linalg.eig(np.matmul(t_tran, t_dist))

        idx = np.where(v == 1)[0]

        print(idx)

    return 0
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

[0.335 0.    0.665 0.   ]


LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [ ]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [ ]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

### Idea of solution

fill in

### Discussion
fill in